In [8]:
def get_json_depth(json_obj, current_path='', depth=0, result=None):
    if result is None:
        result = {}
    
    if isinstance(json_obj, dict):
        for key, value in json_obj.items():
            new_path = f"{current_path}.{key}" if current_path else key
            get_json_depth(value, new_path, depth + 1, result)
    elif isinstance(json_obj, list):
        for index, item in enumerate(json_obj):
            new_path = f"{current_path}[{index}]"
            get_json_depth(item, new_path, depth, result)
    else:  # 基本类型
        result[current_path] = (json_obj, depth)
    
    return result

{'name': ('John', 1),
 'age': (30, 1),
 'address.street': ('21 2nd Street', 2),
 'address.city': ('New York', 2),
 'address.coordinates.latitude': (40.7128, 3),
 'address.coordinates.longitude': (-74.006, 3),
 'hobbies[0]': ('reading', 1),
 'hobbies[1]': ('cycling', 1),
 'hobbies[2].sport': ('swimming', 2)}

In [3]:
import json
from pathlib import Path

menuitemURL = Path(Path.cwd()) / 'menuitemURL.json'

def getRouterName():
    menuitem = json.loads(menuitemURL.read_text(encoding='utf-8'))
    routerName = [key for key in menuitem]
    return routerName
print(getRouterName())

['xm1', 'xm2', 'xm3', 'xm4', 'xm5', 'xm6', '综合', 'xm7', 'xm8']


In [8]:
import pandas as pd
datapath = r'E:\工作文档\开州项目\编号.xlsx'
df = pd.read_excel(datapath)
df = df.fillna('')
TBBH_list = set(df['TBBH'].values)
for tbbh in TBBH_list:
    data = df[df['TBBH'] == tbbh]
    xh_list = list({int(v) for v in data['xh'].values if v != ''})
    xh = 0
    if xh_list:
        xh_max = max(xh_list)
        for i in xh_list:
            data_xh = data[data['xh'] == i]
            for index,row in data_xh.iterrows():
                if xh == 0:
                    xh += 1
                    continue
                else:
                    df.loc[index,'xh'] = xh+ xh_max
                    xh += 1
        xh = 0
    else:
        for index,row in data.iterrows():
            xh += 1
            df.loc[index,'xh'] = xh
        xh = 0

df.to_excel(r"E:\工作文档\开州项目\重新编号.xlsx")

In [ ]:
import geopandas as gpd

gdf = gpd.read_file(r'E:\工作文档\云阳县城市国土空间监测\变更河流水面.shp')
gdf

In [5]:
import pandas as pd
df = pd.read_excel(r'C:\Users\Administrator\Desktop\面积最大.xlsx')
dfdata = pd.DataFrame(columns=df.columns)
CGBH = set(df.CGBH.values)
for v in CGBH:
    CGBH_temp = df[df.CGBH == v]
    Area = CGBH_temp.Area.values
    Area_max = max([value for value in Area])
    res = CGBH_temp[CGBH_temp.Area == Area_max].values[0]
    dfdata.loc[dfdata.shape[0]] = [*res]
dfdata.to_excel(r'E:\工作文档\qita\台账过程\新建文件夹\aa.xlsx', index=False)

In [12]:
a = 420.7801879457542
b = 3366.237731396042
xs = 2
print(b - a*xs)

2524.6773555045334


In [ ]:
11/21/31/40/50/60/70/80
11/21/31/41/51/61/70/80